Go back to the `SolveAiyagari.ipynb` notebook.

# Input

## Structure

In [1]:
struct Params{T<:Real,T1<:Function,T2<:Function,T3<:Function,T4<:Function,
                      T5<:Function,T6<:Function,T7<:Function,T8<:Function,I<:Int64}
    β::T
    α::T
    δ::T
    γ::T
    χ::T
    ϕ::T
    ϵ::T
    κ::T
    ν::T
    tl::T
    tk::T
    Tt::T
    #=
        *** utility functions ***
    =#
    u::T1    # utility function for consumption
    u′::T2 # inverse of u
    inv_u′::T3   # derivative of u
    u′′::T4  # second-order derivative of u 
    v::T5    # disutility function for labor supply
    v′::T6   # derivative of v
    v′′::T7  # second-order derivative of v
    
    # labor supply (comes from labor supply Euler equation)
    l_supply::T8
    
    
    #=
        *** grid for asset choices ***
    =#
    na    ::I # number of grid points
    a_min ::T         # min asset holdings
    a_max ::T         # max asset holdings    
    curv_a  ::T          # curvature of the exponential grid   
    aGrid ::Vector{T} # grid for asset choices

    #=
        *** productivity process ***
    =#
    ny    ::I         #number of states
    ys    ::Vector{T} #productivity states    (States)
    Πy    ::Matrix{T} #Transition matrix (under vectorial format) (Transv)
    Sy    ::Vector{T} #distribution ver y size is ns (disty)
    #yvect ::Array{T,1} #vector of y size is ns*na
end

## Constructor

In [7]:
function Params(
    KsY::T,
    α::T,
    δ::T,
    γ::T,
    ϕ::T,    
    χ::T,
    ϵ::T,
    κ::T,
    ν::T,
    tl::T,
    tk::T,
    Tt::T,
    na::I, 
    a_min::T, 
    a_max::T,     
    curv_a::T,
    ny::I,
    ρy::T, 
    σy::T
    ) where {T<:Real,I<:Int}
    
    #=
        *** Preference parameters ***
    =#
# (rtk + δ) = α*Y/K
# Rt - 1 +δ = α*Y/K
# 1/β - 1 +δ = α*Y/K
# 1/β = 1 + α*Y/K - δ
# 1/β = (1 + α*Y/K - δ)^-1
# 1/β = (1 + α/(K/Y) - δ)^-1

# KSY -> Rt -> β 
    # -> wt




    β = (one(T) + α/KsY - δ)^(-one(T))
    
    Rt = one(T)/β #  before-tax gross interest rate
    wt = (one(T)-α) * ((Rt - (one(T)-δ))/α)^(α/(α-one(T)))
    R = one(T) + (one(T) - tk)*(Rt-one(T))
    # w = tl*wt                                                    ####
    w = (1-tl)*wt                                                  #### good one

    # w = κ*wt^τ/(1+tc)

#     @show κ, wt, τ, 1+tc, w
    χ = (one(T)/(2.8*one(T)))^(one(T)/ϕ)/w
    # χ  = ((1/3.508)^(1/ϕ))/w

    # Tt /= 1+tc
    
    #= 
        *** utility functions ***
    =#
    u = c::T -> (γ ≈ one(T)) ? log.(c) : (c.^(one(T)-γ)-one(T))./(one(T)-γ)::T
    u′ = c::T -> c.^(-γ)::T
    inv_u′ = up::T -> up.^(-one(T)/γ)::T    
    u′′ = c::T -> -γ*c.^(-γ-one(T))::T
    v = l::T -> (one(T)/χ)*(l.^(one(T)+one(T)/ϕ))./(one(T)+one(T)/ϕ)::T # function v
    v′ = l::T -> (one(T)/χ)*l.^(one(T)/ϕ)::T
    v′′ = l::T -> (one(T)/χ)*(one(T)/ϕ)*l.^(one(T)/ϕ-one(T))::T
    
    
    # labor supply (comes from labor supply Euler equation)
    l_supply = (w::T,y::T,u′c::T)->(χ*w*y*u′c)^ϕ::T
    #=
        *** grid for asset choices ***
    =#    
    aGrid = a_min .+ (a_max-a_min)*(range(0.0, 1.0, length=na)).^curv_a
    # aGrid ./= 1+tc    
    #=
        *** productivity process ***
    =#
    ρ   = ρy^0.25 
    #σy  = ((σ2y*(1-ρy^0.5)/(1-ρy^2))^0.5)/(one(T)+ϕ)
    mc    = rouwenhorst(ny, ρy, σy)
    Trans = collect(mc.p')
    Trans[findall(x->x<=5*10^-5,Trans)] .= zero(Trans[1,1])
    for i = 1:ny
        Trans[i,i] += one(Trans[1,1]) - sum(Trans,dims=1)[i]
    end
    
    Sy  = (Trans^100000)[:,1]
    
    endow = exp.(mc.state_values)
    ys = endow./dot(Sy,endow)
    
    return Params{T,typeof(u),typeof(u′),typeof(inv_u′),typeof(u′′),
                    typeof(v),typeof(v′),typeof(v′′),typeof(l_supply),I}(
        β,α,δ,γ,χ,ϕ,ϵ,κ,ν,tl,tk,Tt,
        u,u′,inv_u′,u′′,v,v′,v′′,l_supply,
        na,a_min,a_max,curv_a,aGrid,
        ny,ys,Trans',Sy)
end 

function Params(;
    KsY::T,
    α::T,
    δ::T,
    γ::T,
    ϕ::T,    
    χ::T,
    ϵ::T,
    κ::T,
    ν::T,
    tl::T,
    tk::T,
    Tt::T,
    na::I, 
    a_min::T, 
    a_max::T,     
    curv_a::T,
    ny::I,
    ρy::T, 
    σy::T
    ) where {T<:Real,I<:Int}
    Params(KsY,α,δ,γ,ϕ,χ,ϵ,κ,ν,tl,tk,Tt,na,a_min,a_max,curv_a,ny,ρy,σy)
end

Params

# Output of the Aiyagari model

## Structure 

In [8]:
mutable struct AiyagariSolution{T<:Real,I<:Integer}    
    ga::Matrix{T} # policy function for savings on the asset grid
    gl::Matrix{T} # policy function for labor on the asset grid
    gc::Matrix{T} # policy function for consumption on the asset grid
    Rt::T         # pre-tax gross interest rate
    wt::T         # pre-tax wage rate
    R::T          # post-tax gross interest rate
    w::T          # post-tax wage rate
    A::T          # aggregate savings
    C::T          # aggregate consumption
    K::T          # aggregate capital
    L::T          # aggregate labor supply (in efficient units)
    G::T          # public spending
    Y::T          # GDP
    B::T          # public debt
    transitMat::SparseMatrixCSC{T,I} #
    stationaryDist::Matrix{T} #
    residEuler::Matrix{T} #
end


UndefVarError: UndefVarError: SparseMatrixCSC not defined

## Constructor

In [9]:
function AiyagariSolution(p::Params{T,T1,T2,T3,T4,T5,T6,T7,T8,I}) where {T<:Real,
            T1<:Function,T2<:Function,T3<:Function,T4<:Function,T5<:Function,
            T6<:Function,T7<:Function,T8<:Function,I<:Int64}
    @unpack β,α,δ,γ,χ,ϕ,κ,tl,tk,u′,na,aGrid,ny,ys = p
    Rt = one(T)/β #  before-tax gross interest rate
    
    
    # 1 + rtildek = Rt = 1/β
    # rtildek = 1/β - 1
    # rtildek + δ = 1/β - 1 + δ
    # rtildek + δ = Rt - 1 + δ
    # ζ = 1 = ( (rtildek + δ)/α )^α*(wtilde/(1-α))^(1-α)
    # ζ = 1 = ( (1 + rtildek + δ)/α )^α*(wtilde/(1-α))^(1-α)
    # (wtilde/(1-α))^(α-1) = ( ( rtildek + δ)/α )^α
    # ((wtilde/(1-α))^(α-1))^(1/α) = ( (Rt - 1 + δ)/α )

    wt = (one(T)-α) * ((Rt - (one(T)-δ))/α)^(α/(α-one(T)))



    R = one(T) + (one(T) - tk)*(Rt-one(T))
    # w = κ*wt^τ/(1+tc)
    w = (1-tl)*wt                                                                                   ####

    
    cs = repeat(ys', outer=(na,1))
    function l(y::T)::T
        return (χ*w*(y)*u′(y))^ϕ
    end
    ls = repeat(l.(ys'), outer=(na,1))
    as = repeat(aGrid, outer=(1,ny))
    L  = dot(ys,ls[1,:])
    A  = sum(as)
    return AiyagariSolution{T,I}(as,ls,cs,Rt,wt,R,w,A,A,A,L,
                                    zero(T),A*L,zero(T),
                                    spzeros(T, I, na*ny,na*ny), 
                                    fill(one(T)/(na*ny), na,ny), zeros(T,na,ny))
end

LoadError: LoadError: UndefVarError: @unpack not defined
in expression starting at /Users/alais/Google Drive (alais.martinbaillon@sciencespo.fr)/Post_PhD/Révision_RES/New_code_Julia/Structures_v2.ipynb:4

# Projection

## Structures

We consider three structures for the projection:

* `Allocation_proj` containing the allocations at the truncated history level (as well as sizes and transition matrix)
* `ξs` containing the various ξs
* `Projection` containing the two previous structures

In [ ]:
struct Allocation_proj{I<:Integer,T<:Real}
    S_h::Vector{T}       # size of truncated histories
    Π_h::SparseMatrixCSC{T,I}# transition matrix for histories
    y0_h::Vector{T}      # current productivity levels
    a_beg_h::Vector{T}   # beginning-of-period wealth per history and per capita
    a_end_h::Vector{T}   # end-of-period wealth per history and per capita
    c_h::Vector{T}       # consumption per history and per capita
    l_h::Vector{T}       # labor supply per history and per capita
    ly_h::Vector{T}  # (l y)^τ
    u_h::Vector{T}       # utility of consumption per history and per capita
    u′_h::Vector{T}      # marginal utility of consumption per history and per capita
    u′′_h::Vector{T}     # second-order derivative of utility of consumption per history and per capita
    v_h::Vector{T}       # utility of labor per history and per capita
    v′_h::Vector{T}      # marginal utility of labor per history and per capita
    resid_E_h::Vector{T} # Euler Lagrange multiplier at the history level
    nb_cc_h::I           # nb of credit constrained histories
    ind_cc_h::Vector{I}  # indices of credit constrained histories
end

struct ξs_struct{T<:Real}
    ξu0::Vector{T}
    ξu1::Vector{T}
    ξu2::Vector{T}
    ξuE::Vector{T}
    ξy ::Vector{T}
    ξv0::Vector{T}
    ξv1::Vector{T}
end  

struct Projection{I<:Integer,T<:Real}
    N::I
    Ntot::I
    ind_h::Vector{I}
    allocation_proj::Allocation_proj{I,T}
    ξs::ξs_struct{T}
end

## Constructors

In [ ]:
function Allocation_proj(;S_h::Vector{T},Π_h::SparseMatrixCSC{T,I},y0_h::Vector{T},
        a_beg_h::Vector{T},a_end_h::Vector{T},c_h::Vector{T},l_h::Vector{T},
        ly_h::Vector{T},u_h::Vector{T},u′_h::Vector{T},u′′_h::Vector{T},
        v_h::Vector{T},v′_h::Vector{T},resid_E_h::Vector{T},
        nb_cc_h::I,ind_cc_h::Vector{I}) where{I<:Integer,T<:Real}
    Allocation_proj{I,T}(S_h,Π_h,y0_h,a_beg_h,a_end_h,c_h,l_h,ly_h,u_h,u′_h,u′′_h,
            v_h,v′_h,resid_E_h, nb_cc_h,ind_cc_h)    
end

function ξs_struct(;ξu0::Vector{T},ξu1::Vector{T},ξu2::Vector{T},ξuE::Vector{T},
            ξy::Vector{T},ξv0::Vector{T},ξv1::Vector{T}) where{T<:Real}
    ξs_struct{T}(ξu0,ξu1,ξu2,ξuE,ξy,ξv0,ξv1)
end  

function Projection(;N::I,Ntot::I,ind_h::Vector{I},
        allocation_proj::Allocation_proj{I,T},ξs::ξs_struct{T}) where {I<:Integer,T<:Real}
    Projection{I,T}(N,Ntot,ind_h,allocation_proj,ξs)
end

In [2]:
struct Plan{T <: Real}
    lambdac::Array{T,1}
    lambdact::Array{T,1}
    lambdal::Array{T,1}
    lambdalt::Array{T,1}
    psib::Array{T,1}
    mu::T
    Gamma::T
    Upsilon::T
    Lgamma::T
    # Phi::Array{T,1}
    # CC::Array{T,1} # vector of residuals, it must be 0 to check equations
end

In [2]:
#using SparseArrays

struct Projection_dd{T <: Real,I <: Integer} #be careful the following structure store information by bin (and not per capita). Divide by the size of the bin to have per capita terms    
    N::I #lenght of the truncation
    ns::I #number of states
    Nbin::I #number of bins
    R::T
    w::T
    TT::T
    A::T
    B::T
    states::Array{T,1}
    tk::T
    tl::T
    Transv::Array{T,1}
    Sp::Array{T,1} #size of each bin
    abp::Array{T,1}
    aep::Array{T,1}
    lb::Array{T,1}
    cp::Array{T,1}
    lp::Array{T,1}
    Ucv::Array{T,1} #vector of marginal utilities (before multiplication by xsip) by bin
    Matab::Matrix{T} #transition matrix
    Ucp::Array{T,1}
    Ul::Array{T,1}
    Res::Array{T,1} #check that res is small and Res =  Upvb - β*R*Matab*Upvb
    Ntoi::Array{I,1}
    # CC::Array{T,1}
    Resh::Array{T,1}
    indnc::Array{I,1} #index of non-credit constrained histories
    indcc::Array{I,1} #index of credit constrained histories
    ytype::Array{T,1}
    xsiue::Array{Float64,1}
    xsiy:: Array{Float64,1}
    xsiu0::Array{Float64,1}
    xsiu1::Array{Float64,1}
    xsiu2::Array{Float64,1}
    xsiv0::Array{Float64,1}
    xsiv1::Array{Float64,1}
end

# xsip = xsiE
# xsyn = ξu0
# xsyn1 = ξu1
# xsyn2 = ξu2


# struct Projection_ddd{T <: Real,I <: Integer} #be careful the following structure store information by bin (and not per capita). Divide by the size of the bin to have per capita terms    
#     N::I #lenght of the truncation
#     ns::I #number of states
#     Nbin::I #number of bins
#     R::T
#     w::T
#     TT::T
#     A::T
#     B::T
    # states::Array{T,1}
    # Transv::Array{T,1}
    # Sp::Array{T,1} #size of each bin
    # abp::Array{T,1}
    # aep::Array{T,1}
    # lb::Array{T,1}
    # cp::Array{T,1}
    # Ucv::Array{T,1} #vector of marginal utilities (before multiplication by xsip) by bin
    # Matab::SparseMatrixCSC{T,I} #transition matrix
    # Ucp::Array{T,1}
    # Ul::Array{T,1}
    # Res::Array{T,1} #check that res is small and Res =  Upvb - β*R*Matab*Upvb
    # xsip::Array{T,1}
    # Ntoi::Array{I,1}
    # CC::Array{T,1}
    # indnc::Array{I,1} #index of non-credit constrained histories
    # indcc::Array{I,1} #index of credit constrained histories
    # ytype::Array{T,1}
    # xsyn::Array{Float64,1}
    # xsyn1::Array{Float64,1}
    # xsyn2::Array{Float64,1}
# end

# The function to write the code in Dynare is here

In [ ]:
function Write_Dynare(proj::Projection,solution::AiyagariSolution,params::Params, plan::Plan,calib::String)
    @unpack χ,ϕ,β,α,δ,Tt,u′,na,a_min,aGrid,ny,ys,Sy,ν = params
    @unpack ga,gc,R,w,A,K,C,L = solution



# Res = proj.allocation_proj.u′_h - (params.β*solution.R)*(proj.allocation_proj.Π_h)*proj.allocation_proj.u′_h
#  and xsie ? 


tauopt = -params.Tt/(solution.K^params.α*solution.L^(1-params.α))
Ctot = w*L -A + Tt + R*sum(solution.stationaryDist.*repeat(params.aGrid,1,ny)) # add L here AMB
# Ws = sum(proj.ξs.ξu1 .*proj.allocation_proj.S_h.*proj.allocation_proj.u_h) .+(-Tt)^(params.θ)


# AMB : Attention ici c'est pas propre, à corriger ,  
#Res = test(3,solution,params);

Res = proj.allocation_proj.resid_E_h


eco = Projection_dd(
    proj.N,
    params.ny,
    proj.Ntot,
    solution.R,
    solution.w,
    params.Tt, # pb réglé
    solution.A,
    solution.B,
    params.ys,
    params.tk,
    params.tl,
    collect(vec(params.Πy')),
    proj.allocation_proj.S_h,
    proj.allocation_proj.S_h .*proj.allocation_proj.a_beg_h,#by bin
    proj.allocation_proj.S_h .*proj.allocation_proj.a_end_h,#by bin
    ones(5),
    proj.allocation_proj.S_h .*proj.allocation_proj.c_h,#by bin
    proj.allocation_proj.S_h .*proj.allocation_proj.l_h,# l by bin : I add it here
    proj.allocation_proj.u′_h ./proj.ξs.ξu1, #vector of marginal utilities (before multiplication by xsip) by bin
    Matrix(proj.allocation_proj.Π_h'), # transpose transition matrix !!!!
    proj.allocation_proj.u′_h .*proj.allocation_proj.S_h,
    params.u.(proj.allocation_proj.c_h) .+10,
    Res,  # attention celui-la est faux! vérifier d'ou vient l'erreur
    proj.ind_h,
    proj.allocation_proj.resid_E_h, # celui-la est bon, c'est Resh dans proj_dd et donc dans dynare ; change proj_d here, pb with res AMB
    proj.allocation_proj.ind_cc_h*0,
    proj.allocation_proj.ind_cc_h,
    proj.allocation_proj.y0_h,
    proj.ξs.ξuE,
    proj.ξs.ξy,
    proj.ξs.ξu0,
    proj.ξs.ξu1,
    proj.ξs.ξu2,
    proj.ξs.ξv0,
    proj.ξs.ξv1
    )


# computing the pricing kernel
# M = sum(eco.Ucv.*eco.xsip.*Pl.omega.*eco.Sp)
# à vérifier

M = sum(eco.Ucv.*eco.xsiue.*eco.Sp) ; 

varPSI = sum(Sy.*(ys.^ν)) ;
    
# file = matopen("/Users/alais/Desktop/todynare_Truncation.mat", "w")
file = matopen("../dynamics/todynare_SP_$(calib).mat", "w")
write(file, "Plann", plan) 
write(file, "eco", eco)
write(file, "alphaa", params.α)
write(file, "nu", params.ν)
write(file, "varPSI", varPSI)
write(file, "Ltot", solution.L)
write(file, "kappa", params.κ)
write(file, "betaa", params.β) 
write(file, "phi", params.ϕ)  
write(file, "chi", params.χ) 
write(file, "delta", params.δ)
write(file, "gammaa", params.γ) 
write(file, "epss", params.ϵ)
write(file, "tk", params.tk)
write(file, "tl",params.tl)
write(file, "abar", params.a_min)
write(file, "states", params.ys)
# write(file, "G", -params.Tt)
write(file, "G", solution.G)
write(file, "Ctot", Ctot)
write(file, "M", M)
# write(file, "Ws", Ws)
close(file)

end